In [13]:
from sodapy import Socrata
import requests, zipfile, StringIO
import urllib2
import pandas as pd

In [7]:
client = Socrata("data.cityofnewyork.us", "JbH87Ju4WIAzc35ZUpMxLxFyM")

In [17]:
r = requests.get('http://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_pluto_15v1.zip')
pluto_zip = zipfile.ZipFile(StringIO.StringIO(r.content))

In [20]:
SI = pd.read_csv(pluto_zip.open('SI.csv'))

C:\Anaconda3\envs\py27\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (4,6,7,8,10,11,50,52,53,77,79) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
SI.head()

,Borough,Block,Lot,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,...,YCoord,ZoneMap,ZMCode,Sanborn,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,Version
0,SI,1,10,501,3,2000,31,49,10301,L078,...,171951,21c,,501 017,50101,,0,,1,15v1
1,SI,1,17,501,3,,31,49,10301,L078,...,,21c,,501 017,50101,,0,,1,15v1
2,SI,1,18,501,3,,31,49,10301,L078,...,,21c,,501 017,50101,,0,,1,15v1
3,SI,1,19,501,3,,31,49,10301,L078,...,,21c,,501 017,50101,,0,,1,15v1
4,SI,1,21,501,3,,31,49,10301,L078,...,,21c,,501 017,50101,,0,,1,15v1


In [34]:
mdr = client.get("39m2-a2pz", content_type="csv", limit=20)

In [51]:
mdrdf =pd.DataFrame(mdr)
mdrdf.columns = mdrdf.iloc[0]
mdrdf = mdrdf.reindex(mdrdf.index.drop(0))
mdrdf = mdrdf.convert_objects(convert_numeric=True)

C:\Anaconda3\envs\py27\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [52]:
SI.loc[0]['Lot']

10

In [53]:
mdrdf.head()

,bin,block,boro,boroid,buildingid,communityboard,highhousenumber,housenumber,lastregistrationdate,lot,lowhousenumber,registrationenddate,registrationid,streetcode,streetname,zip
1,2065040,4873,BRONX,2,70619,12,1067,1067,2011-10-15T00:00:00.000,106,1067,2012-10-01T00:00:00.000,222287,28410,EAST 226 STREET,10466
2,4080937,3392,QUEENS,4,661272,5,1924,1924,2015-11-04T00:00:00.000,29,1924,2016-09-01T00:00:00.000,413124,48290,GROVE STREET,11385
3,4042948,1731,QUEENS,4,508388,3,NaN,NaN,2015-11-04T00:00:00.000,42,NaN,2016-09-01T00:00:00.000,410490,19040,97 STREET,11368
4,4544783,1321,QUEENS,4,692605,2,NaN,NaN,2015-11-25T00:00:00.000,61,NaN,2016-09-01T00:00:00.000,916538,61090,ROOSEVELT AVENUE,11377
5,4280961,13069,QUEENS,4,552480,13,NaN,NaN,2015-11-27T00:00:00.000,21,NaN,2016-09-01T00:00:00.000,916385,22820,143 AVENUE,11413
